In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
from fractions import Fraction
import warnings
warnings.filterwarnings('ignore')


In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'Origin'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Window Blinds"""
buckets = """Multiple_Bucket"""

regex_pattern=r'(?i)(USA)|(imported)'

value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [4]:
# df = query_from_file(file_name='Curate_wipe/BusinessIntelligence/Platform Queries/Bucket Attribute Values for All active Strategies.sql', params=params)
df = query_from_file(file_name='query/uncurated.sql', params=params)
df

product_id external_id  \
0       817156024    16471321   
1       817207071    20414559   
2      1054434298    20052409   
3      1054432454    17839187   
4       817455752    20152079   
...           ...         ...   
43764   817456383    40060804   
43765   817158547    20218140   
43766  1133485576    60273611   
43767   817455925    20338480   
43768   817192529    63104691   

                                                                                 product_name  \
0                                  Springs Window Fashions Nickel Finish Holdbacks (Set of 2)   
1      L'Oréal® Paris 8.5 fl. oz. Hair Expert EverCreme Sulfate-Free Deep Nourish Conditioner   
2                                   Matrix Biolage VolumeBloom 33.8 oz. Shampoo for Fine Hair   
3                              Paul Mitchell® 10.14 oz. Smoothing Super Skinny® Daily Shampoo   
4                                    Garden Greens 16 oz. Coconut Oil Paste Pure Extra Virgin   
...                                                                                       ...   
43764                      Boon Twig Grass and Lawn Countertop Drying Rack Accessory in White   
43765                  Burt's Bees® 30-Count Facial Cleansing Towelettes in Cucumber and Sage   
43766                                       Olive Kids Game On Pack 'N Snack Backpack in Blue   
43767  Harmon® Face Values® 8-Count Extra Large Waterproof Antibacterial Knee & Elbow Bandage   
43768                                                          Grandma's Sweethearts Ornament   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       Nickel finish holdbacks are the finishing touch to the Nickel Finish Cafe Drapery Rods and your window treatments. Set of two holdbacks are perfect for holding back window treatments for better visibility and more light. Includes mounting hardware   
1                                                                                           EverCreme Deep Nourish offers penetrating apricot oil to deeply condition without stripping, damaging or drying out your hair. Instead it penetrates moisture and nourishes for healthy soft, silky, shine and texture. EverCreme Sulfate-Free Deep Nourish Conditioner Infuses Essential Nourishment to Revive Hair Sulfate-free, nourishing formula restores lost moisture to each strand with triple action deep conditioners Formulated with Omega-3 and 6, and natural botanicals Triple action hydration for normal to dry hair Sulfate and paraben-free 100% vegan No heavy waxes 8.5 fl. oz. Manufacturer L'Oréal® Paris New York, NY 10017   
2                                                                                                                                                                                 The Biolage VolumeBloom Shampoo removes buildup and cleanses fine hair, giving it a boost of body 

In [5]:
df[df['external_id'].astype(str)=='13876550']

product_id external_id  \
13529   817666425    13876550   

                                            product_name  \
13529  Boneco Air-O-Swiss® Ultrasonic Digital Humidifier   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    long_desc  \
13529  Humidifier generates a micro-fine mist thanks to technology that uses high-frequency vibration. A fan then blows the mist into the room where it immediately evaporates. Ultrasonic humidifier has a built-in hygrostat and an output regulator button A hygienic, pure water mist free of limescale or mineral residues is ensured with the replaceable demineralization cartridge Water is heated to 176° Fahrenheit before it enters the nebulizer chamber when the pre-heating function is used The mist exits at 104° Fahrenheit and helps to prevent the room temperature from falling and keeps the water cleaner Includes demineralization cartridge Humidifier with 1.8-gallon transparent water tank is for rooms up to 650 square feet Three-year manufacturer's warranty Measures approximately 15\" L x 8 3/4\" W x 15 1/2\" H. UL listed Model # AOS7135   

       customer_id attributes                                      buckets  \
13529            5       None  Air Purifiers, Dehumidifiers, & Humidifiers   

      values  
13529   None

In [6]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))


In [7]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

In [8]:
df['matches'].explode().value_counts()

Imported    24470
USA          7398
imported     3533
usa           303
IMported        1
uSa             1
Name: matches, dtype: int64

In [13]:
df[df['matches'].astype(str)!='[]']

product_id external_id  \
131     817158753    20347758   
188     988776741    20523633   
191     988776743    20523640   
206     988762667    20585426   
218    1148362242    20708337   
...           ...         ...   
43761   817647983    65651032   
43762   817465304    46427588   
43763   817481894    40477237   
43766  1133485576    60273611   
43768   817192529    63104691   

                                                                         product_name  \
131                                          Pura D'or® Hair Thinning Therapy Shampoo   
188    Suave® 28 fl. oz. Professionals Coconut Oil Infusion Damage Repair Conditioner   
191        Suave® 28 fl. oz. Professionals Coconut Oil Infusion Damage Repair Shampoo   
206                                       Sun Bum® 1.5 fl. oz. Revitalizing Hair Mask   
218                         Carol's Daughter® 8.45 oz. Coco Creme Coconut Water Spray   
...                                                                               ...   
43761                     Sweet Jojo Designs Cactus Floral 4-Piece Twin Comforter Set   
43762                                      Rockin' Rider Laurel Rocking Horse in Pink   
43763                                My First Memory Foam Toddler Pillow in Soft Pink   
43766                               Olive Kids Game On Pack 'N Snack Backpack in Blue   
43768                                                  Grandma's Sweethearts Ornament   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          long_desc  \
131                                                                                                                                                   This exclusive blend of key active ingredients helps to improve volume, thickness, shine, split ends and a healthy looking scalp. Niacin, Argan Oil, and other natural ingredients help to invigorate and purify the scalp, Invigorating blend of organic extracts and essential nutrients fortifies hair from root to tip, reducing hair loss due to breakage Ultra-nourishing blend of key active naturals including Amla Oil, Argan Oil, Black Cumin Seed, Niacin, Rosemary Oil, Saw Palmetto, Green Tea and an exclusive Botanical Blend work together to effectively treat thinning, damaged, and/or color-treated hair Gentle enough for daily use Helps strengthen, minimize breakage and reduce hair thinning Revives the appearance of damaged and distressed hair, leaving it smoother and healthier looking Fortifies and energizes hair Hibiscus supports strong and healthy-looking hair Organic Argan Oil moisturizes and helps protect hair Organic Rosemary Oil strengthens and moisturizes hair Safe for Damaged and chemically-treated, colored hair No SLS (Sulfates), parabe

In [13]:
df

product_id external_id  \
0      1203386753    20705886   
1      1202465018    17406922   
2      1054420672    20524340   
3      1054432438    17838873   
4      1085627035    20421830   
...           ...         ...   
43764   817647983    65651032   
43765   817158547    20218140   
43766  1133485576    60273611   
43767   817455925    20338480   
43768   817192529    63104691   

                                                                                 product_name  \
0            King C. Gillette Men’s Double Edge Safety Razor with 5 Double Edge Refill Blades   
1                                          Pantene® 6.6 oz. Pro-V Volume Body Boosting Mousse   
2                             Tresemme 22 oz. Pro Collection Keratin Smooth Color Conditioner   
3                                                          Joico Joi 8.5 oz. Firming Hold Gel   
4                                  Garnier® Fructis® 12.5 oz. Fortifying Curl Nourish Shampoo   
...                                                                                       ...   
43764                             Sweet Jojo Designs Cactus Floral 4-Piece Twin Comforter Set   
43765                  Burt's Bees® 30-Count Facial Cleansing Towelettes in Cucumber and Sage   
43766                                       Olive Kids Game On Pack 'N Snack Backpack in Blue   
43767  Harmon® Face Values® 8-Count Extra Large Waterproof Antibacterial Knee & Elbow Bandage   
43768                                                          Grandma's Sweethearts Ornament   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [5]:
# error

In [6]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['Adhesive Mount',
 'Ceiling Mount',
 'Concealed Mount',
 'Corner Mount',
 'Cupboard Mount',
 'Door Mount',
 'Flush Mount',
 'Frame Mount',
 'Freestanding',
 'Hanging',
 'Hinge Mount',
 'Inside Mount',
 'Leaning',
 'Magnetic Mount',
 'Outside Mount',
 'Pole Mount',
 'Post Mount',
 'Pre-Drilled Mounting Holes',
 'Recessed Mount',
 'Roof Mount',
 'Screw Fittings',
 'Semi-Flush Mount',
 'Suction Mount',
 'Tension Mount',
 'Under Cabinet Mount',
 'Wall Mount']

In [7]:
kwp=KeywordProcessor()
print(kwp.add_keywords_from_list(list_values))
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

None


0                     []
1                     []
2        [Outside Mount]
3        [Outside Mount]
4                     []
              ...       
25377                 []
25378                 []
25379          [Hanging]
25380                 []
25381                 []
Name: long_desc, Length: 25382, dtype: object

In [8]:
df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
df=df[df['matches'].astype(str)!='[]']
df['matches'].explode().value_counts()

Hanging                       3240
Wall Mount                    2408
Outside Mount                 1455
Freestanding                   285
Ceiling Mount                   75
Roof Mount                      68
Flush Mount                     49
Leaning                         42
Door Mount                      29
Post Mount                      25
Semi-Flush Mount                20
Recessed Mount                   9
Adhesive Mount                   6
Pre-Drilled Mounting Holes       1
Pole Mount                       1
Name: matches, dtype: int64

In [9]:
# error

In [10]:
matches=df
matches['multi_matches']=matches['matches']
# matches['matches']=matches['Max'].astype(str).apply(lambda quote: quote.replace('.0', ''))
matches.head(1)

product_id external_id  \
14   817331667    65220986   

                                                                product_name  \
14  Bel Air Lighting Argyle 15-Inch Post Mount Outdoor Lantern in Black/Gold   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            long_desc  \
14  Inspired by 18th century carriage lanterns, Bel Air Lighting's Argyle Post Mount Outdoor Lantern captures a vintage look for your traditional home. Its tapered glass panes are topped with a curved, beveled roof and finial for added appeal. Traditional carriage lantern design Hexagonal shape with curved roof and finial Beveled clear glass panes Suitable for dry and damp locations Post mount design Must be hardwired Uses one 60-watt incandescent or CFL or LED equivalent bulb (not included) Steel construction Measures 6\" L x 6\" W x 15\" H 1-year manufacturer's warranty Imported Model 4414BG (Black/Gold); 4414WH (White)   

    customer_id attributes        buckets values                   matches  \
14            5       None  Post Lighting   None  [Post Mount, Post Mount]   

               multi_matches  
14  [Post Mount, Post Mount]

In [11]:
matches['matches'].explode().value_counts()

Hanging                       3240
Wall Mount                    2408
Outside Mount                 1455
Freestanding                   285
Ceiling Mount                   75
Roof Mount                      68
Flush Mount                     49
Leaning                         42
Door Mount                      29
Post Mount                      25
Semi-Flush Mount                20
Recessed Mount                   9
Adhesive Mount                   6
Pre-Drilled Mounting Holes       1
Pole Mount                       1
Name: matches, dtype: int64

In [17]:
# # test_list=pd.DataFrame()
# test_list=matches['matches']
# test_list[0:50]

0                                      [Induction]
2                           [Induction, Induction]
3                           [Induction, Induction]
4                                      [Induction]
5                       [Electric, Gas, Induction]
6                                      [Induction]
7                           [Induction, Induction]
8                                      [Induction]
9                           [Induction, Induction]
10                                     [Induction]
11                          [Induction, Induction]
13                          [Induction, Induction]
18                          [Induction, Induction]
19                          [Induction, Induction]
20                          [Induction, Induction]
21                          [Induction, Induction]
22                          [Induction, Induction]
24                                     [Induction]
25                                     [Induction]
26                             

In [12]:
matches['matches']=[list(set(sublist)) for sublist in matches['matches']]
matches

product_id external_id  \
14      817331667    65220986   
20      817446477    69341595   
22      817489661    17745107   
30      817206738    18292432   
58     1211531703    43374564   
...           ...         ...   
25357   817525737    61461550   
25360   817265905    62489669   
25365      478955    43381869   
25370   988457306    69513631   
25373   817542295    63556278   

                                                                   product_name  \
14     Bel Air Lighting Argyle 15-Inch Post Mount Outdoor Lantern in Black/Gold   
20                          MLB Boston Red Sox Fan Favorite Embossed House Flag   
22                                         MLB Texas Rangers Traditions Pennant   
30                                     MLB New York Yankees Cooperstown Pennant   
58             Real Simple® Cordless Cellular 60.5-Inch x 64-Inch Shade in Sand   
...                                                                         ...   
25357                                           Climbing Man Wall Art in Silver   
25360                         Reasons Why for Him 4-Inch x 6-Inch Picture Frame   
25365           Real Simple® Cordless Cellular 41-Inch x 64-Inch Shade in Shale   
25370       O&O by Olivia & Oliver™ 11-Inch x 17-Inch Metal Wall Frame in Black   
25373                   Private Family Beach 22-Inch x 17.81-Inch Wood Wall Art   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              long_desc  \
14                                                                                                                                                                                                                                                                                                                                                                                                                    Inspired by 18th century carriage lanterns, Bel Air Lighting's Argyle Post Mount Outdoor Lantern captures a vintage look for your traditional home. Its tapered glass panes are topped with a curved, beveled roof and finial for added appeal. Traditional carriage lantern design Hexagonal shape with curved roof and finial Beveled clear glass panes Suitable for dry and damp locations Post mount design Must be hardwired Uses one 60-watt incandescent or CFL or LED equivalent bulb (not included) Steel construction Measures 6\" L x 6\" W x 15\" H 1-year manufacturer's warranty Imported Model 4414BG (Black/Gold); 4414WH (White)   
20                                                                                                                                                                                                                                                                                                                                                                                                                                       Proudly display your pro baseball team pride by hanging this eye-catching MLB Fan Favorite Embossed House Flag. Suitab

In [13]:
# matches['matches'][50:100]

In [14]:
matches[curation_col] = matches['matches'].astype(str)
matches

product_id external_id  \
14      817331667    65220986   
20      817446477    69341595   
22      817489661    17745107   
30      817206738    18292432   
58     1211531703    43374564   
...           ...         ...   
25357   817525737    61461550   
25360   817265905    62489669   
25365      478955    43381869   
25370   988457306    69513631   
25373   817542295    63556278   

                                                                   product_name  \
14     Bel Air Lighting Argyle 15-Inch Post Mount Outdoor Lantern in Black/Gold   
20                          MLB Boston Red Sox Fan Favorite Embossed House Flag   
22                                         MLB Texas Rangers Traditions Pennant   
30                                     MLB New York Yankees Cooperstown Pennant   
58             Real Simple® Cordless Cellular 60.5-Inch x 64-Inch Shade in Sand   
...                                                                         ...   
25357                                           Climbing Man Wall Art in Silver   
25360                         Reasons Why for Him 4-Inch x 6-Inch Picture Frame   
25365           Real Simple® Cordless Cellular 41-Inch x 64-Inch Shade in Shale   
25370       O&O by Olivia & Oliver™ 11-Inch x 17-Inch Metal Wall Frame in Black   
25373                   Private Family Beach 22-Inch x 17.81-Inch Wood Wall Art   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              long_desc  \
14                                                                                                                                                                                                                                                                                                                                                                                                                    Inspired by 18th century carriage lanterns, Bel Air Lighting's Argyle Post Mount Outdoor Lantern captures a vintage look for your traditional home. Its tapered glass panes are topped with a curved, beveled roof and finial for added appeal. Traditional carriage lantern design Hexagonal shape with curved roof and finial Beveled clear glass panes Suitable for dry and damp locations Post mount design Must be hardwired Uses one 60-watt incandescent or CFL or LED equivalent bulb (not included) Steel construction Measures 6\" L x 6\" W x 15\" H 1-year manufacturer's warranty Imported Model 4414BG (Black/Gold); 4414WH (White)   
20                                                                                                                                                                                                                                                                                                                                                                                                                                       Proudly display your pro baseball team pride by hanging this eye-catching MLB Fan Favorite Embossed House Flag. Suitab

In [15]:
# matches

In [29]:
# error

In [18]:
# b_set = set(map(tuple,test_list))  #need to convert the inner lists to tuples so they are hashable
# b = map(list,b_set) #Now convert tuples back into lists (maybe unnecessary?)

In [37]:
# b_set

In [38]:
# def rem_dupes(dup_list): 
#     yooneeks = [] 
#     for elem in dup_list: 
#         if elem not in yooneeks: 
#             yooneeks.append(elem) 
#     return yooneeks

In [39]:
# rem_dupes(test_list[0:5])
# test_list[0:50]

In [40]:
# [list(set(sublist)) for sublist in test_list]

In [41]:
# test_list=test_list.reset_index()
# del test_list['index']
# test_list

In [42]:
# test_list[69]

In [43]:
# sorted(set(test_list))

In [44]:
# new_df=pd.DataFrame()
# for i in range(1):
#     new_df.append(sorted(set(test_list[i])))
# new_df

In [45]:
# test_list=lambda x: [list(set(test_list[i])) for i in test_list]
# test_list

In [46]:
# matches['matches'].astype(str).apply(lambda x: list(set(test_list[i])) for i in range(len(test_list)))

In [47]:
# error

In [48]:
# matches[curation_col] = matches['Max'].astype(str)+' in'
# # matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [49]:
## KWP
# matches[curation_col]=matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [16]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [17]:
looks_good('Bed Bath & Beyond', attribute, df, matches)